In [1]:
# Import Libraries
import os
from glob import glob

import earthpy as et
import earthpy.earthexplorer as etee
import geopandas as gpd 
import geoviews as gv
import holoviews as hv
import pandas as pd
import numpy as np
import rioxarray as rxr
import rioxarray.merge as rxrmerge
import shutil
from bokeh.models import HoverTool


In [2]:
# Define Data Directories for Project
data_dir = os.path.join(et.io.HOME, et.io.DATA_NAME)
dc_dir = os.path.join(data_dir, 'dc-neighborhoods')
ndvi_dir = os.path.join(data_dir, 'dc-green-space', 'processed')

# Check is file directory exists and if not, create it
for file_dir in [dc_dir, ndvi_dir]:
    if not os.path.exists(file_dir):
        os.makedirs()

In [3]:
# Save url for DC Neighborhood boundaries
dc_url = ("https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/"
           "Administrative_Other_Boundaries_WebMercator/MapServer/17/"
           "query?outFields=*&where=1%3D1&f=geojson")

# Get DC Neighborhood Boundaries as a Shapefiles
dc_nbd_gdf = gpd.read_file(dc_url)


In [4]:
# Define path to DC Neighborhood Data
dc_path = os.path.join(dc_dir, 'dc-neighborhood.geojson')



# If the data does not already exist, save data to directory
if not os.path.exists(dc_path):
    # Save url for Chicago Neighborhood boundaries
    dc_url = ("https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/"
            "Administrative_Other_Boundaries_WebMercator/MapServer/17/"
            "query?outFields=*&where=1%3D1&f=geojson"
    )
    # Save Chicago neighborhood data to a file
    gpd.read_file(dc_url).to_file(dc_path)

# Create Geodatabase of Chicago Neighborhood Data
dc_gdf = gpd.read_file(dc_path).set_index("NAME")

# Select Humboldt Park and Lincoln Park Data
neigh_gdf = (
    dc_gdf
    .loc[["Cluster 9", "Cluster 31"]]
)

neigh_gdf

,OBJECTID,WEB_URL,NBH_NAMES,TYPE,GLOBALID,CREATOR,CREATED,EDITOR,EDITED,SHAPE.AREA,SHAPE.LEN,geometry
NAME,,,,,,,,,,,,
Cluster 9,41,http://planning.dc.gov/,"Southwest Employment Area, Southwest/Waterfron...",Original,{0D85A9B0-AAE1-4EE0-A311-793BC87ED3C5},None,None,None,None,0,0,"POLYGON ((-77.02192 38.88757, -77.02742 38.887..."
Cluster 31,33,http://planning.dc.gov/,"Deanwood, Burrville, Grant Park, Lincoln Heigh...",Original,{84876282-0AFE-4553-8F28-958497FC1A4C},None,None,None,None,0,0,"POLYGON ((-76.91322 38.88976, -76.91234 38.890..."


## STEP 3: Download and process raster data

You should have three loops from last week. Convert the operations from each loop into a **function**, starting with the following sample code:

```python
def download_neighborhood_data(name, geometry, start, end)
    """
    Download NAIP raster for a given geometry, start date, and end date

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : str
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data directory.
    """
    <Put your code here>
    return downloader

for neighborhood_name, details in neigh_gdf.interrows():
    download_neighborhood_data(neighborhood_name, details.geometry)

```
One important step of writing function is identifying the **Parameters** and **Returns**. In this case, I have done this for you; for later functions you will need to do this yourself. One way to identify the Parameters is to identify each object or variable used in the code (note that this does not usually include imported classes and functions). 

I am also supplying you with a **docstring** that explains the Parameters and Returns, and specifies their types. Update the docstring if you decide to do something different for your function. When writing docstrings, please follow the [numpy docstring styleguide](https://numpydoc.readthedocs.io/en/latest/format.html#sections)

YOUR TASK:

1. Replace `<Put your code here>` with the download code from last week
2. Open up your summary statistics file, if it exists.
3. Add a **conditional** to your code so that it will skip this download if the summary statistics **already exist** in your summary statistics file!
   
    > HINT: I did this using the `pass` statement, which moves on to the next iteration of the loop. This way you can test if the statistics **do** exist in the file, rather than whether they **do not**. However, there are lots of ways to do this -- do what makes sense to you!
    
4. Test that the code still works for the two-neighborhood `GeoDataFrame`. You should also check that the caching is working (although you may need to wait until you have saved some statistics to do this!)

In [5]:
def download_neighborhood_data(name, geometry, start, end):
    """
    Download NAIP raster for a given geometry, start date, and end date

    Downloads data from the National Agricultural Imagery Program (NAIP)
    given a spatial and temporal extent.
    <citation>

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : str
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data directory.
    """
    print(f'Neighborhood Name: {name}')
    # Create bounding box
    bbox = etee.BBox(*geometry.bounds)
    # Create downloader
    naip_downloader = etee.EarthExplorerDownloader(
        dataset="NAIP", 
        label=name.lower().replace(" ", "-"),
        bbox=bbox,
        start=start,
        end=end,
        store_credential=True)
    # Request and download data
    naip_downloader.submit_download_request()
    naip_downloader.download(override=False)
    return naip_downloader

ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')
if os.path.exists(ndvi_stats_path):
  print('Reading in NDVI Statistics File...')
  ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")
else:
  print('NDVI Statistics File does not exist...')
  ndvi_stats_df = pd.DataFrame()

# # Run to test
# for neighborhood_name, details in neigh_gdf.iterrows():
#     if neighborhood_name in ndvi_stats_df.index:
#       print("Neighborhood stats have already been calculated. Skipping")
#       continue
    
#     downloader = download_neighborhood_data(
#        neighborhood_name, details.geometry, '2021-01-01', '2021-12-31')

Reading in NDVI Statistics File...


YOUR TASK: 

1. Write a function for the loop that loads and merges the arrays.
2. Document your function with a docstring
3. Check that your function works for the Lincoln Park neighborhood

In [6]:
def load_and_merge_arrays(name):
    """
    Load in and merge downloaded arrays
    
    Parameters
    ==========
    name : str
        The name used to label the download
    
    Returns
    =======
    merge_da : rxr.DataArray
        Data array with merged data.
    """
    # Merge data for each neighborhood
    print(f'\nNeighborhood Name: {name}')
    data_path = os.path.join(
        et.io.HOME, et.io.DATA_NAME,
        name.lower().replace(' ', '-'))
    # Define paths to tif data
    tif_paths = glob(os.path.join(data_path, '*.tif'))
    # Load tifs
    das = [rxr.open_rasterio(tp, masked=True) for tp in tif_paths]
    # Merge arrays
    merged_da = rxrmerge.merge_arrays(das)
    return merged_da

# # Run to test
# merged_da = load_and_merge_arrays('Cluster 9')
# merged_da

YOUR TASK:

1. Write a function that computes the NDVI summary statistics and adds them to the statistics file (if the statistics are not already present)
    > HINT: use `mode='a'` to *append* a line to the file instead of writing over existing content
    
2. Document your function with a docstring
3. Check that your function works for the Lincoln Park Neighborhood

In [7]:
# Function to compute NDVI summary statistics
def calculate_ndvi_stats(gdf, da, stats_path, override=False):
    """
    Calculate NDVI summary statistics and save to statistics file
    
    Uses downloaded National Agricultural Imagery Program (NAIP) data.

    Parameters
    ==========
    gdf : [gpd.GeoDataFrame]
        Single row with the neighborhood name and boundary
    da : rxr.DataArray
        Multispectral raster data from NAIP
    stats_path : pathlike
        The path to the statistics file to save results
    """
    name = str(gdf.index[0])
    print(f'\nNeighborhood Name: {name}')

    file_is_empty = True
    if os.path.exists(stats_path):
        print('Stats file exists.')
        stats_df = pd.read_csv(stats_path)
        with open(stats_path, 'r') as stats_file:
            file_is_empty = len(stats_file.read())==0
            print(f'Stats file is empty? {file_is_empty}')
            
            if not file_is_empty:
                if name in list(stats_df.neighborhood) and (not override):
                    print('Stats already calculated. Skipping...')
                    return


    # Create gdf for neighborhood
    reprojected_gdf = gdf.to_crs(da.rio.crs)
    # Crop NAIP data array to the neighborhood
    naip_crop_da = (
        da.rio.clip_box(*reprojected_gdf.total_bounds)
        )
    naip_da = (
        naip_crop_da.rio.clip(reprojected_gdf.geometry)
    )
    
    mode = 'w' if file_is_empty else 'a'
    # Calculate NDVI
    ndvi_da = (da.sel(band=4) - da.sel(band=1)) / (
        da.sel(band=4) + da.sel(band=1)
    )
    print('Writing stats to file')
    file_is_empty = not os.path.exists(stats_path)
    # Calculate summary statistics
    pd.DataFrame(dict(
          neighborhood=[name],
          ndvi_25pctl=[np.nanpercentile(ndvi_da, 25)],
          ndvi_mean=[float(ndvi_da.mean())]
          )).to_csv(stats_path, mode=mode, header=file_is_empty, index=False)

# Run to test
# calculate_ndvi_stats(dc_gdf.loc[['Lincoln Park']], merged_da, ndvi_stats_path)

Putting in all together... YOUR TASK:

1. Create a loop. Start off with just the two neighborhood `GeoDataFrame`.
2. Run each of your functions in the loop, checking that they work. **MAKE SURE YOU INCLUDE CACHING CODE!**
3. Write a line of code at the end of your loop to **delete the raster data files** once you have saved the statistics you want, checking that it works. Use the `shutil.rmtree()` function.
4. Replace the two neighborhood `GeoDataFrame` with the full Chicago `GeoDataFrame`

In [9]:
dc_gdf = dc_gdf.drop('Cluster 45')

KeyError: "['Cluster 45'] not found in axis"

In [12]:
# Redefine NDVI stats path
ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')

# Loop through all Chicago neighborhoods to download data, merge data,
# and calculate NDVI statistics using functions
for neighborhood_name, details in dc_gdf.iterrows():
    if not os.path.exists(ndvi_stats_path):
        print('NDVI statistics file does not exist...')
        ndvi_stats_df = pd.DataFrame()
    else:
        ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col='neighborhood')
        
    if neighborhood_name in ndvi_stats_df.index:
      print("Neighborhood stats have already been calculated. Skipping")
      continue
        
    downloader = download_neighborhood_data(
        neighborhood_name, details.geometry, '2021-01-01', '2021-12-31')
    merged_da = load_and_merge_arrays(neighborhood_name)
    calculate_ndvi_stats(
        dc_gdf.loc[[neighborhood_name]], merged_da, ndvi_stats_path)
    
    shutil.rmtree(downloader.data_dir)

Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood s

Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.



Neighborhood Name: Cluster 46
Stats file exists.
Stats file is empty? False
Writing stats to file
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping
Neighborhood stats have already been calculated. Skipping


In [13]:
ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')

shutil.copyfile(ndvi_stats_path, "ndvi_summary_stats.csv")

'ndvi_summary_stats.csv'

## STEP 4: Plot

YOUR TASK:
1. Join your `GeoDataFrame` of Chicago neighborhoods with your NDVI statistics `DataFrame`
2. Create a Chloropleth plot using one of the statistics for the color scale
3. Write a plot headline and description.

In [ ]:
# Read in NDVI Summary Statistics
ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col="neighborhood")

# Create copy of Neighborhood name for use in hover tool
joined_dc_df = dc_gdf.join(ndvi_stats_df, how="left")
joined_dc_df['name'] = joined_dc_df.index

# Define hover tool for Choropleth
tooltips = [
    ('Neighborhood', '@name'),
    ('NDVI', '@ndvi_mean')
]
hover = HoverTool(tooltips=tooltips)

# Create Choropleth of NDVI Statistics
choropleth = gv.tile_sources.CartoLight * gv.Polygons(
    joined_dc_df,
    vdims=['ndvi_mean', 'name']
).opts(cmap="RdYlGn",
       title="NDVI in Chicago Neighborhoods",
       xaxis=None,
       yaxis=None,
       colorbar=True, 
       colorbar_position="right",
       tools=[hover])

# Save Chloropleth to HTML
hv.save(choropleth, 'choropleth.html')

### NDVI in Chicago Neighborhoods
* NDVI tends to be lower closer to the lake. 
* Areas further from the lake tend to have higher NDVI.
* In general, NDVI tends to be low (under zero) in Chicago.